# Null models

In this notebook we show several basic results concerning the behavior
of quadrangular structural coefficients in standard null models.

## Global $q$-clustering in Erdős–Rényi (ER) random graphs

It is a well known fact that in ER random graphs expected global clustering
is equal to $p$, that is, to the edge exsitence probability.

It can be deduced simply from the fact that for any $2$-path:

```
i -- j -- k
```

the probability that also `i` and `k` are connected is still equal to $p$
as it is constant for all possible edges. Hence, each $2$-path is closed
to make a triangle with probability of exactly $p$ resulting in the global
clustering equal to $p$.

Similarly, for any $3$-path:

```
  i      l
  |      |
  j ---- k
```

probability that it is closed and form a strict quadrangle is equal to
the probability that `i` and `k` as well as `j` and `l` are not connected
which is $(1-p)^2$ times the probability that `i` and `l` are connected
which $p$. So this gives expected quadrangular clustering
(global complementarity) equal to
$p(1-p)^2$. Note that for graphs with low value of $p$ we have that
$(1-p) \approx 1$, so both expected clustering and $q$-clustering
are comparable.

### Simulation

We check the above assertion by simulating $100$ ER random graphs with
$2000$ nodes and different values of $p = .01, .02, .05$.

**NOTE.** This may take a while to execute.

In [1]:
import random
import pandas as pd
import igraph as ig
from tqdm import tqdm
from pathcensus import PathCensus

In [2]:
random.seed(101)

P = (.001, .002, .005, .01)
N = 5000
R = 20

er_null = []

for p in P:
    for _ in tqdm(range(R)):
        graph = ig.Graph.Erdos_Renyi(N, p=p, directed=False)
        paths = PathCensus(graph) 
        index = dict(p=p, q=p*(1-p)**2)
        df = paths.coefs("global")
        for k, v in index.items():
            df[k] = v
        index["q"] = p
        er_null.append(df)

data = pd.concat(er_null, axis=0, ignore_index=True)

100%|██████████| 20/20 [01:57<00:00,  5.88s/it]


In [3]:
sim = data[["p", "sim_g", "sim", "tclust", "tclosure"]] \
    .reset_index() \
    .melt(id_vars=["index", "p"]) \
    .assign(relerr=lambda df: abs(df["value"] - df["p"]) / df["p"])
    
comp = data[[
    "q", "comp_g", "comp", "qclust", "qclosure"
]] \
    .reset_index() \
    .melt(id_vars=["index", "q"]) \
    .assign(relerr=lambda df: abs(df["value"] - df["q"]) / df["q"]) 

### Similarity coefficients vis-a-vis density in ER random graph ($p$)

In [4]:
sim.groupby(["p", "variable"])["value"].describe()

count      mean       std       min       25%       50%  \
p     variable                                                            
0.001 sim        20.0  0.000795  0.000215  0.000405  0.000652  0.000790   
      sim_g      20.0  0.000965  0.000249  0.000457  0.000811  0.000920   
      tclosure   20.0  0.000771  0.000199  0.000382  0.000652  0.000756   
      tclust     20.0  0.000994  0.000335  0.000526  0.000757  0.000963   
0.002 sim        20.0  0.001849  0.000118  0.001588  0.001806  0.001872   
      sim_g      20.0  0.002028  0.000133  0.001748  0.002006  0.002056   
      tclosure   20.0  0.001809  0.000118  0.001551  0.001775  0.001840   
      tclust     20.0  0.002037  0.000127  0.001742  0.001979  0.002078   
0.005 sim        20.0  0.004832  0.000079  0.004725  0.004766  0.004819   
      sim_g      20.0  0.004993  0.000083  0.004894  0.004927  0.004970   
      tclosure   20.0  0.004786  0.000078  0.004690  0.004718  0.004772   
      tclust     20.0  0.004993  0.000081  0.004868  0.004930  0.004979   
0.010 sim        20.0  0.009861  0.000069  0.009747  0.009820  0.009857   
      sim_g      20.0  0.010011  0.000069  0.009881  0.009974  0.010007   
      tclosure   20.0  0.009813  0.000069  0.009695  0.009776  0.009810   
      tclust     20.0  0.010016  0.000070  0.009902  0.009970  0.010012   

                     75%       max  
p     variable                      
0.001 sim       0.000875  0.001328  
      sim_g     0.001088  0.001517  
      tclosure  0.000858  0.001232  
      tclust    0.001100  0.001937  
0.002 sim       0.001903  0.002052  
      sim_g     0.002102  0.002242  
      tclosure  0.001859  0.002009  
      tclust    0.002114  0.002253  
0.005 sim       0.004886  0.005032  
      sim_g     0.005036  0.005195  
      tclosure  0.004837  0.004981  
      tclust    0.005049  0.005205  
0.010 sim       0.009907  0.009996  
      sim_g     0.010058  0.010138  
      tclosure  0.009857  0.009946  
      tclust    0.010064  0.010155

### Complementarity coefficients vis-a-vis $p(1-p)^2$ in ER random graphs

In [5]:
comp.groupby(["q", "variable"])["value"].describe().round(6)

count      mean       std       min       25%       50%  \
q        variable                                                            
0.000998 comp       20.0  0.000777  0.000104  0.000558  0.000713  0.000772   
         comp_g     20.0  0.000992  0.000122  0.000734  0.000910  0.000999   
         qclosure   20.0  0.000764  0.000099  0.000559  0.000705  0.000770   
         qclust     20.0  0.000940  0.000138  0.000653  0.000850  0.000914   
0.001992 comp       20.0  0.001798  0.000043  0.001732  0.001762  0.001800   
         comp_g     20.0  0.001982  0.000048  0.001896  0.001960  0.001974   
         qclosure   20.0  0.001765  0.000042  0.001695  0.001732  0.001765   
         qclust     20.0  0.001960  0.000049  0.001881  0.001926  0.001962   
0.004950 comp       20.0  0.004788  0.000030  0.004733  0.004775  0.004787   
         comp_g     20.0  0.004947  0.000032  0.004893  0.004934  0.004947   
         qclosure   20.0  0.004744  0.000030  0.004691  0.004730  0.004744   
         qclust     20.0  0.004943  0.000032  0.004882  0.004930  0.004942   
0.009801 comp       20.0  0.009656  0.000028  0.009608  0.009635  0.009659   
         comp_g     20.0  0.009803  0.000029  0.009750  0.009780  0.009809   
         qclosure   20.0  0.009610  0.000027  0.009563  0.009590  0.009612   
         qclust     20.0  0.009804  0.000029  0.009751  0.009781  0.009808   

                        75%       max  
q        variable                      
0.000998 comp      0.000875  0.000942  
         comp_g    0.001089  0.001205  
         qclosure  0.000846  0.000921  
         qclust    0.001053  0.001168  
0.001992 comp      0.001838  0.001853  
         comp_g    0.002028  0.002043  
         qclosure  0.001808  0.001818  
         qclust    0.001997  0.002037  
0.004950 comp      0.004812  0.004838  
         comp_g    0.004968  0.005005  
         qclosure  0.004766  0.004793  
         qclust    0.004969  0.004998  
0.009801 comp      0.009671  0.009709  
         comp_g    0.009817  0.009861  
         qclosure  0.009625  0.009662  
         qclust    0.009818  0.009861